In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports

from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, BaseLogger

from GEMS_TCO import kernels_reparam_space_time_cpu_010126 as kernels_reparam_space_time_cpu

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

Load daily data applying max-min ordering

In [2]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

lat_range_input=[-3,-1]      
lon_range_input=[121, 125] 

df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,

lat_range=lat_range_input,   
lon_range=lon_range_input
  
)

In [3]:
daily_aggregated_tensors_dw = [] 
daily_hourly_maps_dw = []      

daily_aggregated_tensors_vecc = [] 
daily_hourly_maps_vecc = []   


for day_index in range(31):
    hour_start_index = day_index * 8
    
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= None,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=False  #keep_exact_loc
    )

    daily_aggregated_tensors_dw.append( day_aggregated_tensor )
    daily_hourly_maps_dw.append( day_hourly_map )

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= ord_mm,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=False  #keep_exact_loc
    )

    daily_aggregated_tensors_vecc.append( day_aggregated_tensor )
    daily_hourly_maps_vecc.append( day_hourly_map )
print(daily_aggregated_tensors_vecc[0].shape)
#print(daily_hourly_maps[0])
nn = daily_aggregated_tensors_vecc[0].shape[0]

torch.Size([22680, 4])


In [15]:
v=0.5
mm_cond_number= 8
nheads = 300
patience, factor = 5, 0.5


# full likelihood

In [16]:
# --- 1. 환경 설정 ---
# Full Likelihood는 CPU에서 BLAS(MKL) 최적화를 사용하는 것이 수치적으로 더 안정적일 수 있습니다.
DEVICE = torch.device("cpu") 
print(f"Using device: {DEVICE}")

# L-BFGS 설정 (FL은 수렴이 더 민감할 수 있으므로 MAX_STEPS를 조절하세요)
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 2   
LBFGS_HISTORY_SIZE = 100   
LBFGS_MAX_EVAL = 100       

# --- 2. 최적화 루프 ---
days_list = [2]

for day_idx in days_list:
    print(f'\n{"="*40}')
    print(f'--- Full Likelihood Processing for Day {day_idx+1} ---')
    print(f'{"="*40}')

    # 데이터 로드 (FL은 베키아용 nns_map 등이 필요 없음)
    daily_hourly_map_fl = daily_hourly_maps_vecc[day_idx]
    daily_aggregated_tensor_fl = daily_aggregated_tensors_vecc[day_idx]

    # 초기값 설정 (기존과 동일)
    init_sigmasq   = 13.059
    init_range_lat = 0.2
    init_range_lon = 0.25
    init_advec_lat = 0.0218
    init_range_time = 1.5
    init_advec_lon = -0.1689
    init_nugget    = 0.247
    
    # Phi 재파라미터화
    init_phi2 = 1.0 / init_range_lon
    init_phi1 = init_sigmasq * init_phi2
    init_phi3 = (init_range_lon / init_range_lat)**2
    init_phi4 = (init_range_lon / init_range_time)**2

    initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                    np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

    # 파라미터 텐서 생성 (반드시 dtype=torch.float64 권장)
    params_list = [
        torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
        for val in initial_vals
    ]

    # --- 💥 Full Likelihood 클래스 인스턴스 생성 ---
    # 이전에 정의한 FullLikelihoodCPU 클래스를 사용합니다.
    # nns_map, mm_cond_number, nheads 인자가 빠집니다.
    model_instance = kernels_reparam_space_time_cpu.FullLikelihoodCPU(
            smooth = v,
            input_map = daily_hourly_map_fl,
            aggregated_data = daily_aggregated_tensor_fl
        )

    # --- 💥 L-BFGS 옵티마이저 설정 ---
    # 최적화 메서드는 기존 SpatioTemporalModel에서 상속받은 것을 사용하거나 직접 선언
    optimizer_fl = torch.optim.LBFGS(
        params_list, 
        lr=LBFGS_LR, 
        max_iter=LBFGS_MAX_EVAL, 
        history_size=LBFGS_HISTORY_SIZE,
        line_search_fn="strong_wolfe" # FL은 곡률이 가파를 수 있어 라인 서치 추가 권장
    )

    print(f"\n--- Starting Phase: Full Likelihood Optimization (Day {day_idx+1}) ---")
    start_time = time.time()
    
    # --- 💥 최적화 실행 (Closure 정의) ---
    def closure():
        optimizer_fl.zero_grad()
        params = torch.stack(params_list)
        loss = model_instance.compute_nll(params) # 클래스 내 NLL 함수 호출
        loss.backward()
        return loss

    for i in range(LBFGS_MAX_STEPS):
        loss = optimizer_fl.step(closure)
        with torch.no_grad():
            current_loss = loss.item()
            print(f"Step {i+1}/{LBFGS_MAX_STEPS} | Loss: {current_loss:.6f}")
            if torch.isnan(loss):
                print("Warning: Loss is NaN, stopping...")
                break

    end_time = time.time()
    
    # 결과 해석 (기존 helper 함수 사용)
    final_raw = [p.item() for p in params_list]
    interpretable = model_instance._convert_raw_params_to_interpretable(final_raw)
    
    print(f"\nFull Likelihood finished in {end_time - start_time:.2f}s.")
    print(f"Final Interpretable Params: {interpretable}")

Using device: cpu

--- Full Likelihood Processing for Day 3 ---
✅ Full Likelihood initialized with 22680 points.

--- Starting Phase: Full Likelihood Optimization (Day 3) ---
Step 1/2 | Loss: 1.064576
Step 2/2 | Loss: 1.046486

Full Likelihood finished in 8236.42s.
Final Interpretable Params: {'sigma_sq': 13.33556708063782, 'range_lon': 0.2626306030189747, 'range_lat': 0.23251439212476346, 'range_time': 1.4963416581821842, 'advec_lat': -0.09606783899984583, 'advec_lon': -0.20754696131728878, 'nugget': 4.275661732987106e-08}


Using device: cpu

irregular grid (original)
========================================
--- Full Likelihood Processing for Day 3 ---
========================================
✅ Full Likelihood initialized with 22680 points.

--- Starting Phase: Full Likelihood Optimization (Day 3) ---
Step 1/2 | Loss: 1.069034
Step 2/2 | Loss: 1.054918

Full Likelihood finished in 2298.03s.
Final Interpretable Params: {'sigma_sq': 14.240365190909174, 'range_lon': 0.32155366170834, 'range_lat': 0.28451293128038607, 'range_time': 1.783983438665154, 'advec_lat': -0.10064979654054186, 'advec_lon': -0.19632058367668134, 'nugget': 0.3588147517482075}

regular grid


Using device: cpu

========================================
--- Full Likelihood Processing for Day 3 ---
========================================
✅ Full Likelihood initialized with 22680 points.

--- Starting Phase: Full Likelihood Optimization (Day 3) ---
Step 1/2 | Loss: 1.064576
Step 2/2 | Loss: 1.046486

Full Likelihood finished in 8236.42s.
Final Interpretable Params: {'sigma_sq': 13.33556708063782, 'range_lon': 0.2626306030189747, 'range_lat': 0.23251439212476346, 'range_time': 1.4963416581821842, 'advec_lat': -0.09606783899984583, 'advec_lon': -0.20754696131728878, 'nugget': 4.275661732987106e-08}

4,4

    # 초기값 설정 (기존과 동일)
    init_sigmasq   = 13.059
    init_range_lat = 0.2
    init_range_lon = 0.25
    init_advec_lat = 0.0218
    init_range_time = 1.5
    init_advec_lon = -0.1689
    init_nugget    = 0.247
========================================
--- Full Likelihood Processing for Day 3 ---
========================================
✅ Full Likelihood initialized with 9280 points.

--- Starting Phase: Full Likelihood Optimization (Day 3) ---
Step 1/2 | Loss: 1.582280
Step 2/2 | Loss: 1.556743

irregular 

Full Likelihood finished in 309.40s.
Final Interpretable Params: {'sigma_sq': 12.768700531979968, 'range_lon': 0.49839139178737574, 'range_lat': 0.38739297037364306, 'range_time': 2.743756962827003, 'advec_lat': -0.08105917546965641, 'advec_lon': -0.2439350666438791, 'nugget': 2.876802721846206}

--- Starting Phase: Full Likelihood Optimization (Day 3) ---
Step 1/2 | Loss: 1.581733
Step 2/2 | Loss: 1.556079

regular


Full Likelihood finished in 308.51s.
Final Interpretable Params: {'sigma_sq': 12.795534206579426, 'range_lon': 0.5092607675638082, 'range_lat': 0.3979372586860229, 'range_time': 2.7997211043259043, 'advec_lat': -0.08101618106442411, 'advec_lon': -0.24427939930448753, 'nugget': 2.9203892557743343}

# 계산 

In [1]:
import torch
import numpy as np

# --- 1. 설정 ---
DEVICE = torch.device("cpu")
DTYPE = torch.float64
T_STEPS = 8  # 시뮬레이션 코드와 동일하게 8

print(f"Using device: {DEVICE}")

# --- 2. True Parameters ---
true_vals = {
    'sigmasq': 13.059, 'range_lat': 0.154, 'range_lon': 0.195,
    'range_time': 1.0, 'advec_lat': 0.042, 'advec_lon': -0.1689, 'nugget': 0.247
}

# --- 3. Estimated Parameters (Regular / Irregular) ---
# [사용자 입력] 아까 분석했던 추정값들을 그대로 사용합니다.
# (만약 새로운 결과가 있다면 여기 숫자만 바꿔주세요)

# Regular Grid 추정값 (Case 1 기준)
reg_est_list = [
    12.186636803, 0.136710721, 0.173858427, 0.927052480, 
    -0.036711323, -0.176327904, 0.168845631
]
# Irregular Grid 추정값 (Case 1 기준)
irr_est_list = [
    12.270305048, 0.323816638, 0.403037715, 1.786609867, 
    -0.033519437, -0.180801754, 3.275174630
]

reg_params = torch.tensor(reg_est_list, device=DEVICE, dtype=DTYPE, requires_grad=True)
irr_params = torch.tensor(irr_est_list, device=DEVICE, dtype=DTYPE, requires_grad=True)


# --- 4. 공분산 함수 (동일) ---
def get_cov_from_physical(params, coords):
    sigmasq, r_lat, r_lon, r_time, adv_lat, adv_lon, nugget = params

    phi2 = 1.0 / r_lon
    phi1 = sigmasq * phi2
    phi3 = (r_lon / r_lat)**2
    phi4 = (r_lon / r_time)**2
    
    x_lat, x_lon, x_time = coords[:, 0], coords[:, 1], coords[:, 2]
    u_lat = x_lat - adv_lat * x_time
    u_lon = x_lon - adv_lon * x_time
    
    d_lat = u_lat.unsqueeze(1) - u_lat.unsqueeze(0)
    d_lon = u_lon.unsqueeze(1) - u_lon.unsqueeze(0)
    d_t   = x_time.unsqueeze(1) - x_time.unsqueeze(0)
    
    dist_sq = (d_lat**2 * phi3) + (d_lon**2) + (d_t**2 * phi4)
    dist = torch.sqrt(dist_sq + 1e-8)
    
    cov = (phi1/phi2) * torch.exp(-dist * phi2)
    cov = cov + torch.eye(cov.shape[0], device=DEVICE, dtype=DTYPE) * (nugget + 1e-6)
    return cov


# ==============================================================================
# [수정됨] 5. Grid Setup (시뮬레이션 코드와 100% 일치시킴)
# ==============================================================================
# 요청 범위: Lat [-1, -3], Lon [121, 125]
target_lat_start = -1.0 
target_lat_end   = -3.0
target_lon_start = 121.0
target_lon_end   = 125.0

step_lat = -0.044
step_lon = 0.063

# [핵심 수정] 시뮬레이션 코드의 로직 복사 (Rounding 포함)
lats = torch.arange(target_lat_start, target_lat_end - 0.0001, step_lat, device=DEVICE, dtype=DTYPE)
lats = torch.round(lats * 10000) / 10000

lons = torch.arange(target_lon_start, target_lon_end + 0.0001, step_lon, device=DEVICE, dtype=DTYPE)
lons = torch.round(lons * 10000) / 10000

times = torch.arange(0.0, float(T_STEPS), 1.0, device=DEVICE, dtype=DTYPE) # 0.0 ~ 7.0

# Meshgrid 생성
grid_lat, grid_lon, grid_time = torch.meshgrid(lats, lons, times, indexing='ij')
coords = torch.stack([grid_lat.flatten(), grid_lon.flatten(), grid_time.flatten()], dim=1)

print(f"\n[Validation Grid Sync Check]")
print(f"  Lat Range: {lats[0].item()} ~ {lats[-1].item()} (Size: {len(lats)})")
print(f"  Lon Range: {lons[0].item()} ~ {lons[-1].item()} (Size: {len(lons)})")
print(f"  Time Steps: {len(times)}")
print(f"  Total Validation Points: {coords.shape[0]}")
print("-" * 60)


# --- 6. True Matrix Build (동일) ---
true_params_tensor = torch.tensor([
    true_vals['sigmasq'], true_vals['range_lat'], true_vals['range_lon'], true_vals['range_time'],
    true_vals['advec_lat'], true_vals['advec_lon'], true_vals['nugget']
], device=DEVICE, dtype=DTYPE)

# 메모리 절약을 위해 no_grad 안에서 생성
with torch.no_grad():
    Sigma_true = get_cov_from_physical(true_params_tensor, coords)


# --- 7. Bias Calculation Function (동일) ---
def calc_bias(est_tensor):
    Sigma_est = get_cov_from_physical(est_tensor, coords)
    
    try:
        L = torch.linalg.cholesky(Sigma_est.detach())
        Sigma_est_inv = torch.cholesky_inverse(L)
    except:
        Sigma_est_inv = torch.linalg.pinv(Sigma_est.detach())

    loss1 = 0.5 * torch.logdet(Sigma_est)
    grads1 = torch.autograd.grad(loss1, est_tensor, create_graph=True)[0]
    
    M = Sigma_est_inv @ Sigma_true @ Sigma_est_inv
    loss2 = 0.5 * torch.trace(M @ Sigma_est)
    grads2 = torch.autograd.grad(loss2, est_tensor, create_graph=False)[0]
    
    return (-grads1 + grads2).detach().numpy()


# --- 8. 실행 및 출력 ---
print("Calculating Analytical Bias (Synced Grid)...")
bias_reg = calc_bias(reg_params)
bias_irr = calc_bias(irr_params)

names = ['sigmasq', 'range_lat', 'range_lon', 'range_time', 'adv_lat', 'adv_lon', 'nugget']
true_list = true_params_tensor.tolist()
reg_vals = reg_params.detach().tolist()
irr_vals = irr_params.detach().tolist()

print(f"\n{'='*100}")
print(f"{'Param':<12} | {'True':<8} | {'Reg Est':<10} | {'Reg Bias':<10} | {'Irr Est':<10} | {'Irr Bias':<10}")
print(f"{'-'*100}")

for i, name in enumerate(names):
    print(f"{name:<12} | {true_list[i]:<8.3f} | {reg_vals[i]:<10.4f} | {bias_reg[i]:<10.4f} | {irr_vals[i]:<10.4f} | {bias_irr[i]:<10.4f}")
print(f"{'='*100}")

Using device: cpu

[Validation Grid Sync Check]
  Lat Range: -1.0 ~ -2.98 (Size: 46)
  Lon Range: 121.0 ~ 124.969 (Size: 64)
  Time Steps: 8
  Total Validation Points: 23552
------------------------------------------------------------
Calculating Analytical Bias (Synced Grid)...


: 

스파스

In [13]:
import torch
import numpy as np

# ==============================================================================
# 1. 환경 설정
# ==============================================================================
DEVICE = torch.device("cpu")
DTYPE = torch.float64
T_STEPS = 8  # 시간 편향 검증 필수

print(f"Using device: {DEVICE}")

# --- True Parameters (Ground Truth) ---
true_vals = {
    'sigmasq': 13.059, 'range_lat': 0.154, 'range_lon': 0.195,
    'range_time': 1.0, 'advec_lat': 0.042, 'advec_lon': -0.1689, 'nugget': 0.247
}

# ==============================================================================
# 2. 이미지 데이터 입력 (Transcription from Screenshots)
# ==============================================================================

# [Iteration 1 Data]
iter1_reg = [12.186636803, 0.136710721, 0.173858427, 0.927052480, -0.036711323, -0.176327904, 0.168845631]
iter1_irr = [12.270305048, 0.323816638, 0.403037715, 1.786609867, -0.033519437, -0.180801754, 3.275174630]

# [Iteration 2 Data] - *Looks Promising!*
iter2_reg = [13.388171927, 0.151010171, 0.194674887, 0.984971704, -0.047754729, -0.162044208, 0.142254978]
iter2_irr = [15.050859502, 0.377967901, 0.474854695, 2.043965860, -0.049417309, -0.160316221, 3.203429650]

# 묶어서 처리
iterations_data = [
    ("Iteration 1", iter1_reg, iter1_irr),
    ("Iteration 2", iter2_reg, iter2_irr),
]

# ==============================================================================
# 3. 검증용 Grid Setup (Downsampled for Speed)
# ==============================================================================
target_lat_start = -1.0 
target_lat_end   = -3.0
target_lon_start = 121.0
target_lon_end   = 125.0
step_lat = -0.044
step_lon = 0.063

# Full Grid 생성
full_lats = torch.arange(target_lat_start, target_lat_end - 0.0001, step_lat, device=DEVICE, dtype=DTYPE)
full_lats = torch.round(full_lats * 10000) / 10000
full_lons = torch.arange(target_lon_start, target_lon_end + 0.0001, step_lon, device=DEVICE, dtype=DTYPE)
full_lons = torch.round(full_lons * 10000) / 10000

# Downsampling [::3]
lats = full_lats[::3]
lons = full_lons[::3]
times = torch.arange(0.0, float(T_STEPS), 1.0, device=DEVICE, dtype=DTYPE)

grid_lat, grid_lon, grid_time = torch.meshgrid(lats, lons, times, indexing='ij')
coords = torch.stack([grid_lat.flatten(), grid_lon.flatten(), grid_time.flatten()], dim=1)

print(f"Grid Setup: {coords.shape[0]} points (Downsampled)")


# ==============================================================================
# 4. 분석 엔진 (Covariance & Bias Logic)
# ==============================================================================
def get_cov_from_physical(params, coords):
    sigmasq, r_lat, r_lon, r_time, adv_lat, adv_lon, nugget = params
    phi2 = 1.0 / r_lon
    phi1 = sigmasq * phi2
    phi3 = (r_lon / r_lat)**2
    phi4 = (r_lon / r_time)**2
    
    x_lat, x_lon, x_time = coords[:, 0], coords[:, 1], coords[:, 2]
    u_lat = x_lat - adv_lat * x_time
    u_lon = x_lon - adv_lon * x_time
    
    d_lat = u_lat.unsqueeze(1) - u_lat.unsqueeze(0)
    d_lon = u_lon.unsqueeze(1) - u_lon.unsqueeze(0)
    d_t   = x_time.unsqueeze(1) - x_time.unsqueeze(0)
    
    dist_sq = (d_lat**2 * phi3) + (d_lon**2) + (d_t**2 * phi4)
    dist = torch.sqrt(dist_sq + 1e-8)
    
    cov = (phi1/phi2) * torch.exp(-dist * phi2)
    cov = cov + torch.eye(cov.shape[0], device=DEVICE, dtype=DTYPE) * (nugget + 1e-6)
    return cov

# True Matrix Build
true_params_tensor = torch.tensor([
    true_vals['sigmasq'], true_vals['range_lat'], true_vals['range_lon'], true_vals['range_time'],
    true_vals['advec_lat'], true_vals['advec_lon'], true_vals['nugget']
], device=DEVICE, dtype=DTYPE)

with torch.no_grad():
    Sigma_true = get_cov_from_physical(true_params_tensor, coords)

def calc_bias(est_list):
    est_tensor = torch.tensor(est_list, device=DEVICE, dtype=DTYPE, requires_grad=True)
    Sigma_est = get_cov_from_physical(est_tensor, coords)
    
    try:
        L = torch.linalg.cholesky(Sigma_est.detach())
        Sigma_est_inv = torch.cholesky_inverse(L)
    except:
        Sigma_est_inv = torch.linalg.pinv(Sigma_est.detach())

    # Term 1
    loss1 = 0.5 * torch.logdet(Sigma_est)
    grads1 = torch.autograd.grad(loss1, est_tensor, create_graph=True)[0]
    
    # Term 2
    M = Sigma_est_inv @ Sigma_true @ Sigma_est_inv
    loss2 = 0.5 * torch.trace(M @ Sigma_est)
    grads2 = torch.autograd.grad(loss2, est_tensor, create_graph=False)[0]
    
    return (-grads1 + grads2).detach().numpy()


# ==============================================================================
# 5. 실행 및 결과 출력
# ==============================================================================
names = ['sigmasq', 'range_lat', 'range_lon', 'range_time', 'adv_lat', 'adv_lon', 'nugget']
true_list = true_params_tensor.tolist()

for title, reg_data, irr_data in iterations_data:
    print(f"\n\n{'#'*80}")
    print(f"   Analyzing {title}")
    print(f"{'#'*80}")
    
    bias_reg = calc_bias(reg_data)
    bias_irr = calc_bias(irr_data)
    
    print(f"\n{'Param':<12} | {'True':<8} | {'Reg Est':<10} | {'Reg Bias':<10} | {'Irr Est':<10} | {'Irr Bias':<10}")
    print(f"{'-'*80}")
    
    for i, name in enumerate(names):
        print(f"{name:<12} | {true_list[i]:<8.3f} | {reg_data[i]:<10.4f} | {bias_reg[i]:<10.1f} | {irr_data[i]:<10.4f} | {bias_irr[i]:<10.1f}")

    print("-" * 80)
    
    # 간단 진단 로직
    reg_nugget = reg_data[6]
    irr_nugget = irr_data[6]
    

print( 13.059 / 0.154, 12.1866/0.1367, 12.2703/0.3238 )

print(13.059/0.154, 13.3882/0.1510, 15.0509/0.3779 )

Using device: cpu
Grid Setup: 2816 points (Downsampled)


################################################################################
   Analyzing Iteration 1
################################################################################

Param        | True     | Reg Est    | Reg Bias   | Irr Est    | Irr Bias  
--------------------------------------------------------------------------------
sigmasq      | 13.059   | 12.1866    | 3.6        | 12.2703    | 5.2       
range_lat    | 0.154    | 0.1367     | 191.8      | 0.3238     | -78.4     
range_lon    | 0.195    | 0.1739     | 117.4      | 0.4030     | -93.0     
range_time   | 1.000    | 0.9271     | -53.4      | 1.7866     | -5.3      
adv_lat      | 0.042    | -0.0367    | 503.6      | -0.0335    | 388.7     
adv_lon      | -0.169   | -0.1763    | 30.9       | -0.1808    | 28.8      
nugget       | 0.247    | 0.1688     | 3.7        | 3.2752     | 10.9      
-----------------------------------------------------------------

# reg  irreg inter - reg again

In [14]:
import torch
import numpy as np

import torch
import numpy as np
import pandas as pd

# ==============================================================================
# 1. 환경 설정
# ==============================================================================
DEVICE = torch.device("cpu")
DTYPE = torch.float64
T_STEPS = 8 

print(f"Using device: {DEVICE}")

# --- True Parameters (Ground Truth) ---
true_vals = {
    'sigmasq': 13.059, 'range_lat': 0.154, 'range_lon': 0.195,
    'range_time': 1.0, 'advec_lat': 0.042, 'advec_lon': -0.1689, 'nugget': 0.247
}

# ==============================================================================
# 2. 데이터 입력 (스크린샷 기반 Transcription)
# ==============================================================================

# [Iteration 1]
iter1_irr = [
    26.621504752677602, 0.6166100129579051, 0.8074627335843512, 3.334144386668055, 
    -0.03532621226119369, -0.17826168637776335, 0.012677796352343495
]
iter1_reg = [
    16.15106702302292, 0.3206055413389437, 0.41861461876277206, 1.817646875672847, 
    -0.03452362900067263, -0.17898069123692426, 3.669356306300312e-08
]

# [Iteration 2] - ★ NEW ★
iter2_irr = [
    47.08171757757822, 1.1077961684407176, 1.457807566689824, 5.9857702718295345, 
    -0.049919460308025014, -0.15982457836489355, 0.021624284775190367
]
iter2_reg = [
    22.939301979725716, 0.455845958811843, 0.6001056365545133, 2.6031922104357585, 
    -0.04833189200068252, -0.16323148256176445, 1.8336971711857282e-09
]

iterations_data = [
    ("Iteration 1", iter1_reg, iter1_irr),
    ("Iteration 2", iter2_reg, iter2_irr),
]

# ==============================================================================
# 3. Grid Setup (Downsampled for Speed)
# ==============================================================================
target_lat_start = -1.0 
target_lat_end   = -3.0
target_lon_start = 121.0
target_lon_end   = 125.0
step_lat = -0.044
step_lon = 0.063

full_lats = torch.arange(target_lat_start, target_lat_end - 0.0001, step_lat, device=DEVICE, dtype=DTYPE)
full_lats = torch.round(full_lats * 10000) / 10000
full_lons = torch.arange(target_lon_start, target_lon_end + 0.0001, step_lon, device=DEVICE, dtype=DTYPE)
full_lons = torch.round(full_lons * 10000) / 10000

# Downsampling [::3]
lats = full_lats[::3]
lons = full_lons[::3]
times = torch.arange(0.0, float(T_STEPS), 1.0, device=DEVICE, dtype=DTYPE)

grid_lat, grid_lon, grid_time = torch.meshgrid(lats, lons, times, indexing='ij')
coords = torch.stack([grid_lat.flatten(), grid_lon.flatten(), grid_time.flatten()], dim=1)

print(f"Validation Grid Points: {coords.shape[0]}")


# ==============================================================================
# 4. 분석 엔진 (Bias Calc)
# ==============================================================================
def get_cov_from_physical(params, coords):
    sigmasq, r_lat, r_lon, r_time, adv_lat, adv_lon, nugget = params
    phi2 = 1.0 / r_lon
    phi1 = sigmasq * phi2
    phi3 = (r_lon / r_lat)**2
    phi4 = (r_lon / r_time)**2
    
    x_lat, x_lon, x_time = coords[:, 0], coords[:, 1], coords[:, 2]
    u_lat = x_lat - adv_lat * x_time
    u_lon = x_lon - adv_lon * x_time
    
    d_lat = u_lat.unsqueeze(1) - u_lat.unsqueeze(0)
    d_lon = u_lon.unsqueeze(1) - u_lon.unsqueeze(0)
    d_t   = x_time.unsqueeze(1) - x_time.unsqueeze(0)
    
    dist_sq = (d_lat**2 * phi3) + (d_lon**2) + (d_t**2 * phi4)
    dist = torch.sqrt(dist_sq + 1e-8)
    
    cov = (phi1/phi2) * torch.exp(-dist * phi2)
    cov = cov + torch.eye(cov.shape[0], device=DEVICE, dtype=DTYPE) * (nugget + 1e-6)
    return cov

# True Matrix Build
true_params_tensor = torch.tensor([
    true_vals['sigmasq'], true_vals['range_lat'], true_vals['range_lon'], true_vals['range_time'],
    true_vals['advec_lat'], true_vals['advec_lon'], true_vals['nugget']
], device=DEVICE, dtype=DTYPE)

with torch.no_grad():
    Sigma_true = get_cov_from_physical(true_params_tensor, coords)

def calc_bias(est_list):
    est_tensor = torch.tensor(est_list, device=DEVICE, dtype=DTYPE, requires_grad=True)
    Sigma_est = get_cov_from_physical(est_tensor, coords)
    
    try:
        L = torch.linalg.cholesky(Sigma_est.detach())
        Sigma_est_inv = torch.cholesky_inverse(L)
    except:
        Sigma_est_inv = torch.linalg.pinv(Sigma_est.detach())

    loss1 = 0.5 * torch.logdet(Sigma_est)
    grads1 = torch.autograd.grad(loss1, est_tensor, create_graph=True)[0]
    
    M = Sigma_est_inv @ Sigma_true @ Sigma_est_inv
    loss2 = 0.5 * torch.trace(M @ Sigma_est)
    grads2 = torch.autograd.grad(loss2, est_tensor, create_graph=False)[0]
    
    return (-grads1 + grads2).detach().numpy()


# ==============================================================================
# 5. 실행 및 결과 출력
# ==============================================================================
names = ['sigmasq', 'range_lat', 'range_lon', 'range_time', 'adv_lat', 'adv_lon', 'nugget']
true_list = true_params_tensor.tolist()

for title, reg_data, irr_data in iterations_data:
    print(f"\n\n{'='*100}")
    print(f"   Analyzing {title}")
    print(f"{'='*100}")
    
    bias_reg = calc_bias(reg_data)
    bias_irr = calc_bias(irr_data)
    
    print(f"\n{'Param':<12} | {'True':<8} | {'Forced Reg':<12} | {'Bias (Reg)':<10} | {'Orig Irr':<12} | {'Bias (Irr)':<10}")
    print(f"{'-'*100}")
    
    for i, name in enumerate(names):
        print(f"{name:<12} | {true_list[i]:<8.3f} | {reg_data[i]:<12.4f} | {bias_reg[i]:<10.1f} | {irr_data[i]:<12.4f} | {bias_irr[i]:<10.1f}")

    # 간단 진단
    r_lat_true = true_list[1]
    r_lat_reg = reg_data[1]
    r_lat_irr = irr_data[1]
    

# --- True Values ---
true_vals = {'sigmasq': 13.059, 'range_lat': 0.154}

# --- Iteration 2 Results (from Screenshot) ---
# Forced Regular
reg_sigmasq = 22.939
reg_range_lat = 0.456

# Original Irregular
irr_sigmasq = 47.082
irr_range_lat = 1.108

# --- 1. Calculate Ratios ---
# Microergodic Parameter c = sigma^2 / range
true_ratio = true_vals['sigmasq'] / true_vals['range_lat']
reg_ratio = reg_sigmasq / reg_range_lat
irr_ratio = irr_sigmasq / irr_range_lat

# --- 2. Print Analysis ---

print( true_ratio, 16.1511/0.3206, 26.6215/0.6166 )
print( true_ratio, reg_ratio, irr_ratio )

Using device: cpu
Validation Grid Points: 2816


   Analyzing Iteration 1

Param        | True     | Forced Reg   | Bias (Reg) | Orig Irr     | Bias (Irr)
----------------------------------------------------------------------------------------------------
sigmasq      | 13.059   | 16.1511      | 43.5       | 26.6215      | 35.5      
range_lat    | 0.154    | 0.3206       | -931.6     | 0.6166       | -692.9    
range_lon    | 0.195    | 0.4186       | -702.2     | 0.8075       | -519.5    
range_time   | 1.000    | 1.8176       | -54.2      | 3.3341       | -30.6     
adv_lat      | 0.042    | -0.0345      | 595.7      | -0.0353      | 627.5     
adv_lon      | -0.169   | -0.1790      | 33.7       | -0.1783      | 28.3      
nugget       | 0.247    | 0.0000       | 171.0      | 0.0127       | 263.3     


   Analyzing Iteration 2

Param        | True     | Forced Reg   | Bias (Reg) | Orig Irr     | Bias (Irr)
----------------------------------------------------------------------------

# vecc

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Global L-BFGS Settings
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 7      # 10 to 20  
LBFGS_HISTORY_SIZE = 100   
LBFGS_MAX_EVAL = 100       # line search from 50 to 80
       
DELTA_LAT, DELTA_LON = 0.044, 0.063 
LAT_COL, LON_COL, VAL_COL, TIME_COL = 0, 1, 2, 3

days_list = [2]

# --- 2. Run optimization loop over pre-loaded data ---

for day_idx in days_list:  # 0-based

    print(f'\n{"="*40}')
    print(f'--- Starting Processing for Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'{"="*40}')

    # Assuming data access is correct
    daily_hourly_map_dw = daily_hourly_maps_dw[day_idx]
    daily_aggregated_tensor_dw = daily_aggregated_tensors_dw[day_idx]

    daily_hourly_map_vecc = daily_hourly_maps_vecc[day_idx]
    daily_aggregated_tensor_vecc = daily_aggregated_tensors_vecc[day_idx]

    if isinstance(daily_aggregated_tensor_vecc, torch.Tensor):
        daily_aggregated_tensor_vecc = daily_aggregated_tensor_vecc.to(DEVICE)


    init_sigmasq   = 13.059
    init_range_lat = 0.2
    init_range_lon = 0.25
    init_advec_lat = 0.0218
    init_range_time = 1.5
    init_advec_lon = -0.1689
    init_nugget    = 0.247

    #init_sigmasq   = 13.059
    #init_range_lat = 0.154 
    #init_range_lon = 0.195
    #init_advec_lat = 1e-8
    #init_range_time = 1e-8
    #init_advec_lon = 1e-8
    #init_nugget    = 0.247
    
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    # Create Initial Parameters (Float64, Requires Grad)
    initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                    np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

    params_list = [
        torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
        for val in initial_vals
    ]

    # --- 💥 Instantiate the GPU Batched Class ---
    # NOTE: Ensure fit_vecchia_lbfgs is the NEW class we defined
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map_vecc,
            aggregated_data = daily_aggregated_tensor_vecc,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )


    # --- 💥 Set L-BFGS Optimizer ---
    optimizer_vecc = model_instance.set_optimizer(
                params_list,     
                lr=LBFGS_LR,            
                max_iter=LBFGS_MAX_EVAL,        
                history_size=LBFGS_HISTORY_SIZE 
            )

    print(f"\n--- Starting Phase 2: Vecchia Optimization (Day {day_idx+1}) ---")
    start_time = time.time()
    
    # --- 💥 Call the Batched Fit Method ---
    # REMOVED: model_instance.matern_cov_aniso_STABLE_log_reparam
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer_vecc,
            # covariance_function argument is GONE
            max_steps=LBFGS_MAX_STEPS, 
            grad_tol=1e-7
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Vecchia Optimization finished in {epoch_time:.2f}s. Results: {out}")


Using device: cpu

--- Starting Processing for Day 3 (2024-07-3) ---

--- Starting Phase 2: Vecchia Optimization (Day 3) ---
Pre-computing Batched Tensors (Padding Strategy)... Done. Heads: 2400, Batched Tails: 142608
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/7 / Loss: 1.154766 ---
  Param 0: Value=4.1854, Grad=-4.886735384828939e-07
  Param 1: Value=1.5657, Grad=3.3554310193209536e-08
  Param 2: Value=0.0847, Grad=-4.1048802831790323e-07
  Param 3: Value=-4.2799, Grad=4.923925520427055e-08
  Param 4: Value=-0.0649, Grad=-9.755465208913239e-07
  Param 5: Value=-0.2317, Grad=8.480952916056772e-07
  Param 6: Value=-15.4450, Grad=1.4169959112953606e-09
  Max Abs Grad: 9.755465e-07
------------------------------
--- Step 2/7 / Loss: 1.146785 ---
  Param 0: Value=4.1854, Grad=-4.886735384828939e-07
  Param 1: Value=1.5657, Grad=3.3554310193209536e-08
  Param 2: Value=0.0847, Grad=-4.1048802831790323e-07
  Param 3: Value=-4.2799, Grad=4.923925520427055e-08
  Param 4: Valu

day 3 
```mm:8, nheads:0 3.3min + sin + cos```

Final Interpretable Params: {'sigma_sq': 5.729729026727045, 'range_lon': 0.0036587431849657167, 'range_lat': 0.0012094699144324097, 'range_time': 0.03315069403922441, 'advec_lat': 0.0006871610223819161, 'advec_lon': 0.7768481448824361, 'nugget': 0.4047672260885547}
Vecchia Optimization finished in 206.37s. Results: [7.356303821961211, 5.610635582627295, 2.2138750399607745, -4.407887912768792, 0.0006871610223819161, 0.7768481448824361, -0.9044431275065533, 1.3942282339249539]

```mm:8, nheads:0 3.3min ```

Final Interpretable Params: {'sigma_sq': 13.52176155987271, 'range_lon': 0.14804554772619838, 'range_lat': 0.1144343090263059, 'range_time': 1.2325348810769514, 'advec_lat': -0.041359274162119944, 'advec_lon': -0.2538392052470705, 'nugget': 0.9374975398710232}
Vecchia Optimization finished in 205.28s. Results: [4.514535652675534, 1.9102352976581036, 0.515038086528947, -4.238616450537956, -0.041359274162119944, -0.2538392052470705, -0.06454114527858945, 1.4484173299536962]

```mm:8, nheads:5 4min ```

```mm:8, nheads:30 4min ```
Final Interpretable Params: {'sigma_sq': 13.669364586514357, 'range_lon': 0.15083264731249288, 'range_lat': 0.11662213059603312, 'range_time': 1.2627057882308315, 'advec_lat': -0.04101681075075879, 'advec_lon': -0.25248042684726085, 'nugget': 0.9611024244021281}
Vecchia Optimization finished in 204.66s. Results: [4.50674152004807, 1.8915843527250762, 0.5144637421408024, -4.249682444376359, -0.04101681075075879, -0.25248042684726085, -0.039674294627593726, 1.4485796682053353]


```mm:15, nheads:300 8min ```

Final Interpretable Params: {'sigma_sq': 15.987012980327377, 'range_lon': 0.19554905112684073, 'range_lat': 0.1520424726426302, 'range_time': 1.7346813870286135, 'advec_lat': -0.04175957765041507, 'advec_lon': -0.24798155593579485, 'nugget': 1.2705783726841615}

``` debiased whittle ```
Best Run Loss: 5.757 (after 4 steps)
Final Parameters (Natural Scale): sigmasq: 17.4925, range_lat: 0.1538, range_lon: 0.1915, range_time: 1.6396, advec_lat: -0.0530, advec_lon: -0.2803, nugget: 1.5813
Final Parameters (Phi Scale)    : phi1: 91.3249, phi2: 5.2208, phi3: 1.5503, phi4: 0.0136, advec_lat: -0.0530, advec_lon: -0.2803, nugget: 1.5813
Final Parameters (Raw Log Scale): log_phi1: 4.5144, log_phi2: 1.6527, log_phi3: 0.4384, log_phi4: -4.2942, advec_lat: -0.0530, advec_lon: -0.2803, log_nugget: 0.4583


day2

```mm:8, nheads:0 4min ```

Final Interpretable Params: {'sigma_sq': 8.366564121506674, 'range_lon': 0.1549748061133354, 'range_lat': 0.11153205307539416, 'range_time': 1.0734237460346854, 'advec_lat': -0.00623351472675741, 'advec_lon': -0.24543605291094764, 'nugget': 1.6400890271121227}
Vecchia Optimization finished in 227.53s. Results: [3.988736017479325, 1.8644927164784315, 0.6579010827747122, -3.870692038510152, -0.00623351472675741, -0.24543605291094764, 0.49475052518720436, 1.3441941337277519]

```mm:8, nheads:5 4min ```

Final Interpretable Params: {'sigma_sq': 8.423172104750531, 'range_lon': 0.15700473815159813, 'range_lat': 0.11301047728007582, 'range_time': 1.0894565926043982, 'advec_lat': -0.005855656240283135, 'advec_lon': -0.24334669105231177, 'nugget': 1.6523327736126328}
Vecchia Optimization finished in 329.65s. Results: [3.982465786618215, 1.851479294779673, 0.6575909009911699, -3.87431665562769, -0.005855656240283135, -0.24334669105231177, 0.502188091624637, 1.3442811993310593]


```mm:8, nheads:10 5min ```

Final Interpretable Params: {'sigma_sq': 8.434624560416324, 'range_lon': 0.15742622658981514, 'range_lat': 0.11331541468641343, 'range_time': 1.092870211035363, 'advec_lat': -0.005776271785214885, 'advec_lon': -0.2429149020688287, 'nugget': 1.6548849040774298}
Vecchia Optimization finished in 262.90s. Results: [3.9811435383315907, 1.8487983330555655, 0.657563470385232, -3.8752115791546777, -0.005776271785214885, -0.2429149020688287, 0.5037314620466141, 1.3442947956599645]

```mm:15 nheads:300 ```

Final Interpretable Params: {'sigma_sq': 12.138675796215404, 'range_lon': 0.2853604451458316, 'range_lat': 0.20648779278033955, 'range_time': 2.2060040135925485, 'advec_lat': 0.001727014542248498, 'advec_lon': -0.22293591604378923, 'nugget': 2.087514406427404}

Vecchia Optimization finished in 953.12s. Results: [3.7503988799281887, 1.2540021779879755, 0.6470236104112204, -4.090369836431771, 0.001727014542248498, -0.22293591604378923, 0.7359740790016484, 1.3452577850426004]


========================================
--- Starting Processing for Day 1 (2024-07-1) ---
========================================

```mm:7 nheads:0 5 min ```

Final Interpretable Params: {'sigma_sq': 10.569605281314743, 'range_lon': 0.14200359882303407, 'range_lat': 0.11161968489461344, 'range_time': 0.8715661717453963, 'advec_lat': 0.0242886130403664, 'advec_lon': -0.15954080680753288, 'nugget': 6.138438989319248e-07}
Vecchia Optimization finished in 1053.64s. Results: [4.309885333762307, 1.951902877877842, 0.48150995753408793, -3.628878779078998, 0.0242886130403664, -0.15954080680753288, -14.30352517739001, 1.240425409122721]

```mm:12, nheads:0 5min ```

Final Interpretable Params: {'sigma_sq': 10.488076410034935, 'range_lon': 0.14073044003890806, 'range_lat': 0.11039052166423442, 'range_time': 0.835231003054122, 'advec_lat': 0.02363100822784711, 'advec_lon': -0.16357402304597604, 'nugget': 6.728098082283848e-07}
Vecchia Optimization finished in 414.83s. Results: [4.3111480230549795, 1.9609089911476811, 0.4856440243355976, -3.561724098239924, 0.02363100822784711, -0.16357402304597604, -14.211803150171008, 1.240218304649 4564]

```mm:8, nheads:0 5min ```

------------------------------
Final Interpretable Params: {'sigma_sq': 10.488076417803729, 'range_lon': 0.14073044016388103, 'range_lat': 0.1103905217342916, 'range_time': 0.835231004195405, 'advec_lat': 0.023631008151489937, 'advec_lon': -0.1635740224244672, 'nugget': 6.727973102027563e-07}
Vecchia Optimization finished in 222.84s. Results: [4.311148022907675, 1.9609089902596504, 0.4856440248423989, -3.5617240991967183, 0.023631008151489937, -0.1635740224244672, -14.211821726210712, 1.240218304649 459]

```mm:8, nheads:10 5min ```
Final Interpretable Params: {'sigma_sq': 10.558430388457724, 'range_lon': 0.14179260996001697, 'range_lat': 0.1112165306452909, 'range_time': 0.8446228888601491, 'advec_lat': 0.023328002993195814, 'advec_lon': -0.16258995452127456, 'nugget': 4.637894119371829e-07}
Vecchia Optimization finished in 1152.10s. Results: [4.310314411953833, 1.9533897821800137, 0.4857729383037368, -3.56904949113498, 0.023328002993195814, -0.16258995452127456, -14.583835241326524, 1.2403595466586055]

```mm:8, nheads:80 5min ```

Final Interpretable Params: {'sigma_sq': 11.006086403532715, 'range_lon': 0.14860068457265108, 'range_lat': 0.11650340473006518, 'range_time': 0.9077858958066443, 'advec_lat': 0.021852814885628166, 'advec_lon': -0.15862268076456867, 'nugget': 7.109280446538787e-07}
Vecchia Optimization finished in 292.76s. Results: [4.304940969085704, 1.9064925398944794, 0.48668448271634956, -3.6194916282784857, 0.021852814885628166, -0.15862268076456867, -14.156694615282996, 1.2410108632965844]

```mm:8, nheads:200 4min ```

Final Interpretable Params: {'sigma_sq': 12.156526241674566, 'range_lon': 0.16620409318999774, 'range_lat': 0.1301807912751779, 'range_time': 1.077543051896279, 'advec_lat': 0.020739697904739146, 'advec_lon': -0.16128425389734136, 'nugget': 2.436005746521788e-06}
Vecchia Optimization finished in 240.63s. Results: [4.292404933576785, 1.7945387687653933, 0.48858464754993547, -3.738444532638281, 0.020739697904739146, -0.16128425389734136, -12.925150849120314, 1.2417995126905985]

```mm:8, nheads:400 4min ```

------------------------------
Final Interpretable Params: {'sigma_sq': 13.08618055898487, 'range_lon': 0.18072835038101287, 'range_lat': 0.14158038290565716, 'range_time': 1.1895069654662571, 'advec_lat': 0.02010820528296312, 'advec_lon': -0.16111328907142078, 'nugget': 0.005886870024760588}
Vecchia Optimization finished in 230.72s. Results: [4.282316955935752, 1.7107602017089585, 0.4882548888256034, -3.7685982164921903, 0.02010820528296312, -0.16111328907142078, -5.1350308275285474, 1.2419952634078875]

```mm:15 nheads 300 12 min```
Final Interpretable Params: {'sigma_sq': 12.768798755341088, 'range_lon': 0.17558306854693478, 'range_lat': 0.13749362701220857, 'range_time': 1.1605600864511632, 'advec_lat': 0.020485081623863546, 'advec_lon': -0.16411004529108067, 'nugget': 9.430158191707344e-06}


Vecchia Optimization finished in 691.05s. Results: [4.286647620915056, 1.7396430230054256, 0.48906937789270644, -3.7770914894896723, 0.020485081623863546, -0.16411004529108067, -11.57159768609298, 1.2419278421000428]


